In [1]:
# repo root
%cd /work

/work


In [2]:
!ls 

LICENSE    adapters  examples  paper	       scripts	tests
README.md  docker    out       pyproject.toml  src


In [3]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [4]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [5]:
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [6]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [7]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [22]:
# Fig2 run
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --taus 0.2,0.4,0.6,0.8,0.9

[run_fig2] OK
  out: /work/paper/source_data/PANCAN_TP53_v1/out


In [23]:
import llm_pathway_curator.pipeline as p
print("pipeline file:", p.__file__)
print("run_meta default:", p.RunConfig.run_meta_name)

pipeline file: /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py
run_meta default: run_meta.json


In [24]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours

audit_log.tsv		    report.jsonl	       tau_0.40
claims.proposed.tsv	    report.md		       tau_0.60
distilled.tsv		    risk_coverage.tsv	       tau_0.80
distilled.with_modules.tsv  run_meta.json	       tau_0.90
evidence.normalized.tsv     sample_card.resolved.json  term_gene_edges.tsv
modules.tsv		    tau_0.20		       term_modules.tsv


In [25]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20
report   True 3285
audit    True 3361
modules  True 13767
edges    True 88967
terms    True 3188
distilled True 37821
risk     True 235
card     True 414
meta     True 3487

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (3, 30)
[report] decisions: {'PASS': 3}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'cancer', 'comparison', 'tau', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'claim.claim_id', 'claim.entity', 'claim.direction', 'claim.context_keys', 'claim.evidence_ref.module_id', 'claim.evidence_ref.gene_ids']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comp

In [26]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80
report   True 3285
audit    True 3361
modules  True 13767
edges    True 88967
terms    True 3188
distilled True 37821
risk     True 235
card     True 414
meta     True 3487

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (3, 30)
[report] decisions: {'PASS': 3}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'cancer', 'comparison', 'tau', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'claim.claim_id', 'claim.entity', 'claim.direction', 'claim.context_keys', 'claim.evidence_ref.module_id', 'claim.evidence_ref.gene_ids']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comp

In [27]:
# 選択が上位の超安定termだけを拾ってるから tau 0.2↔0.8 で見た目が変わらない
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80/distilled.tsv"
df=pd.read_csv(p, sep="\t")
print("n", len(df), "cols", list(df.columns)[:20])
if "term_survival" in df.columns:
    s=pd.to_numeric(df["term_survival"], errors="coerce")
    print("term_survival describe:\n", s.describe())
    print("unique (head):", sorted(s.dropna().unique())[:20])
else:
    print("NO term_survival column")

n 50 cols ['term_id', 'term_name', 'source', 'stat', 'qval', 'direction', 'evidence_genes', 'pval', 'is_summary', 'group_id', 'evidence_genes_str', 'qval_source', 'qval_provided', 'pval_provided', 'is_valid', 'invalid_reason', 'n_evidence_genes', 'masked_genes_count', 'term_row_id', 'term_uid']
term_survival describe:
 count    50.000000
mean      0.933438
std       0.063513
min       0.734375
25%       0.906250
50%       0.953125
75%       0.984375
max       1.000000
Name: term_survival, dtype: float64
unique (head): [np.float64(0.734375), np.float64(0.78125), np.float64(0.8125), np.float64(0.828125), np.float64(0.875), np.float64(0.890625), np.float64(0.90625), np.float64(0.921875), np.float64(0.9375), np.float64(0.953125), np.float64(0.96875), np.float64(0.984375), np.float64(1.0)]


In [28]:
# A) 0.8未満のtermが “そもそも eligible で落ちてる” のか？
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80/distilled.tsv"
df=pd.read_csv(p, sep="\t")
s=pd.to_numeric(df["term_survival"], errors="coerce")
print("n<0.8", (s<0.8).sum())
print(df.loc[s<0.8, ["term_id","term_survival","is_valid","invalid_reason","n_evidence_genes"]].sort_values("term_survival"))


n<0.8 2
                     term_id  term_survival  is_valid  invalid_reason  \
19     HALLMARK_ANGIOGENESIS       0.734375      True             NaN   
3   HALLMARK_NOTCH_SIGNALING       0.781250      True             NaN   

    n_evidence_genes  
19                 7  
3                  9  


In [29]:
a="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80/audit_log.tsv"
df=pd.read_csv(a, sep="\t")

# 全体
print("audit n", len(df))
print("status counts:\n", df["status"].value_counts(dropna=False))
print("term_survival_agg describe:\n", pd.to_numeric(df["term_survival_agg"], errors="coerce").describe())

# 低survivalのterm_idが監査対象に存在するか
low = {"HALLMARK_ANGIOGENESIS","HALLMARK_NOTCH_SIGNALING"}
hit = df[df["term_id"].isin(low)].copy()
print("hits", len(hit))
if len(hit):
    cols=["term_id","term_survival_agg","term_survival","eligible","preselect_tau_gate","keep_term","keep_reason","status","audit_notes","tau_used"]
    print(hit[cols].sort_values(["term_survival_agg","term_id"]))


audit n 3
status counts:
 status
PASS    3
Name: count, dtype: int64
term_survival_agg describe:
 count    3.000000
mean     0.989583
std      0.018042
min      0.968750
25%      0.984375
50%      1.000000
75%      1.000000
max      1.000000
Name: term_survival_agg, dtype: float64
hits 0


In [30]:
import llm_pathway_curator, inspect
from llm_pathway_curator.pipeline import RunConfig, run_pipeline

print("LLMPC:", llm_pathway_curator.__file__)
print("RunConfig:", inspect.getsourcefile(RunConfig))
print("run_pipeline:", inspect.getsourcefile(run_pipeline))


LLMPC: /opt/LLM-PathwayCurator/src/llm_pathway_curator/__init__.py
RunConfig: /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py
run_pipeline: /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py


In [31]:
import inspect, os
import llm_pathway_curator.pipeline as p

print("PIPE:", inspect.getsourcefile(p))
src = inspect.getsource(p.run_pipeline)
# k_eff周りが見つかる場所を出す
for i, line in enumerate(src.splitlines(), 1):
    if "k_eff" in line or "LLMPATH_K_CLAIMS" in line or "k_claims" in line and "inputs" in line:
        print(f"{i:04d}: {line}")

PIPE: /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py
0199:         k_eff, k_meta = _resolve_k_claims(cfg, card)
0202:             "k_effective": int(k_eff),
0203:             "k_env_raw": os.environ.get("LLMPATH_K_CLAIMS", ""),
0210:             k=int(k_eff),


In [32]:
import json, pathlib
base = pathlib.Path("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20")
m = json.loads((base/"run_meta.json").read_text())
print("run_id:", m.get("run_id"))
print("config.k_claims:", m.get("config", {}).get("k_claims"))
print("inputs.claims:", m.get("inputs", {}).get("claims"))
print("cmd:", m.get("cmd"), "status:", m.get("status"), "step:", m.get("step"))

run_id: 1768960618_3e733ab77e
config.k_claims: None
inputs.claims: None
cmd: run status: ok step: report_jsonl


In [33]:
import json, pathlib
base = pathlib.Path("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20")
m = json.loads((base/"run_meta.json").read_text())
print("cwd:", m.get("inputs", {}).get("cwd", m.get("config", {}).get("cwd", "")))
print("config.evidence_table:", m.get("config", {}).get("evidence_table"))
print("config.sample_card:", m.get("config", {}).get("sample_card"))

cwd: 
config.evidence_table: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv
config.sample_card: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/sample_card.tau.json


In [34]:
!nl -ba /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py | sed -n '185,230p'


   185	            raise ValueError(f"card.k_claims() must be >= 1 (got {k_card})")
   186	        return int(k_card), {"k_source": "card", "k_cfg": None, "k_env": "", "k_card": str(k_card)}
   187	
   188	    # 4) fallback
   189	    return 3, {"k_source": "fallback", "k_cfg": None, "k_env": "", "k_card": ""}
   190	
   191	
   192	# -------------------------
   193	# Fig2 v4 stress helpers (pipeline-owned)
   194	# -------------------------
   195	_NA_TOKENS = {"", "na", "nan", "none", "NA"}
   196	
   197	
   198	def _is_na_scalar(x: Any) -> bool:
   199	    if x is None:
   200	        return True
   201	    if isinstance(x, (list, tuple, set, dict)):
   202	        return False
   203	    try:
   204	        v = pd.isna(x)
   205	        return bool(v) if isinstance(v, bool) else False
   206	    except Exception:
   207	        return False
   208	
   209	
   210	def _parse_ids(x: Any) -> list[str]:
   211	    if _is_na_scalar(x):
   212	        return []
   213	    if isinstance

In [35]:
import json, pathlib
p = pathlib.Path("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/run_meta.json")
m = json.loads(p.read_text())
print(json.dumps(m, indent=2, sort_keys=True)[:4000])

{
  "artifacts": {
    "audit_log_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/audit_log.tsv",
    "claims_proposed_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/claims.proposed.tsv",
    "distilled_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/distilled.tsv",
    "distilled_with_modules_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/distilled.with_modules.tsv",
    "evidence_normalized_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/evidence.normalized.tsv",
    "modules_tsv": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/modules.tsv",
    "report_dir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20",
    "report_jsonl": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/report.jsonl",
    "sample_card_resolved_json": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/sample_card.resolved.json",
    "term_gene_edges_t

In [36]:
import llm_pathway_curator, llm_pathway_curator.pipeline as p, inspect, os
print("PACKAGE_ROOT =", os.path.dirname(llm_pathway_curator.__file__))
print("PIPELINE_FILE =", p.__file__)
print("RUNCONFIG_FIELDS =", list(getattr(p, "RunConfig").__dataclass_fields__.keys()))
print("HAS_resolve_k_claims =", hasattr(p, "_resolve_k_claims"))
print("PIPELINE_SNIP =")
src = inspect.getsource(p._resolve_k_claims) if hasattr(p, "_resolve_k_claims") else "(no _resolve_k_claims)"
print(src[:400])

import llm_pathway_curator.pipeline as p, inspect, hashlib, pathlib
path = pathlib.Path(p.__file__)
txt = path.read_text(encoding="utf-8")
print("PIPELINE_FILE =", p.__file__)
print("PIPELINE_SHA256_12 =", hashlib.sha256(txt.encode("utf-8")).hexdigest()[:12])
print("HAS__resolve_k_claims =", "_resolve_k_claims" in txt)
print("HAS_inputs_claims_write =", 'meta["inputs"]["claims"]' in txt)
print("HAS_k_eff_assign =", "k_eff, k_meta = _resolve_k_claims" in txt)
print("RUN_PIPELINE_LINE_NO =", [i for i,l in enumerate(txt.splitlines(),1) if "def run_pipeline" in l][:1])
print("K_BLOCK_LINE_NO =", [i for i,l in enumerate(txt.splitlines(),1) if "k_eff, k_meta = _resolve_k_claims" in l][:1])

import json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20/run_meta.json"
m=json.load(open(p))
print("run_id", m.get("run_id"))
print("status/step", m.get("status"), m.get("step"))
print("config keys", sorted(list(m.get("config", {}).keys())))
print("has inputs.claims?", "claims" in (m.get("inputs", {}) or {}))
print("inputs.claims", (m.get("inputs", {}) or {}).get("claims"))

import llm_pathway_curator.pipeline as p
import inspect
print("pipeline file:", p.__file__)
print("RunConfig fields:", list(p.RunConfig.__dataclass_fields__.keys()))
print("has _resolve_k_claims:", hasattr(p, "_resolve_k_claims"))
print("resolve_k_claims src head:")
print(inspect.getsource(p._resolve_k_claims).splitlines()[0] if hasattr(p,"_resolve_k_claims") else "NO")


PACKAGE_ROOT = /opt/LLM-PathwayCurator/src/llm_pathway_curator
PIPELINE_FILE = /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py
RUNCONFIG_FIELDS = ['evidence_table', 'sample_card', 'outdir', 'force', 'seed', 'run_meta_name', 'tau', 'k_claims']
HAS_resolve_k_claims = True
PIPELINE_SNIP =
def _resolve_k_claims(cfg: RunConfig, card: SampleCard) -> tuple[int, dict[str, Any]]:
    """
    Single source-of-truth for k_claims used by claim selection.
    Priority:
      1) cfg.k_claims (explicit run override; Fig2/CLI should set this)
      2) env LLMPATH_K_CLAIMS
      3) card.k_claims() if available
      4) fallback 3

    Returns:
      (k_effective, meta_dict)
    """
    # 1) cfg

PIPELINE_FILE = /opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py
PIPELINE_SHA256_12 = cddd52c8f81f
HAS__resolve_k_claims = True
HAS_inputs_claims_write = True
HAS_k_eff_assign = True
RUN_PIPELINE_LINE_NO = [618]
K_BLOCK_LINE_NO = [816]
run_id 1768960618_3e733ab77e
status/step ok report_js

In [37]:
import importlib
import llm_pathway_curator.pipeline as p
p = importlib.reload(p)

print("has _resolve_k_claims:", hasattr(p, "_resolve_k_claims"))

has _resolve_k_claims: True


In [38]:
import json
m=json.load(open("paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80/run_meta.json"))
print("config.k_claims =", m.get("config", {}).get("k_claims"))
print("LLMPATH_K_CLAIMS(raw) =", (m.get("inputs", {}).get("claims", {}) or {}).get("k_env_raw"))
print("inputs.claims =", m.get("inputs", {}).get("claims"))


config.k_claims = None
LLMPATH_K_CLAIMS(raw) = None
inputs.claims = None


In [41]:
!which llm-pathway-curator
!python -c "import llm_pathway_curator.pipeline as p; print(p.__file__)"


/opt/conda/envs/py/bin/llm-pathway-curator
/opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py


In [31]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [43]:
!python paper/scripts/run_fig2_pipeline.py \
    --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
    --variants ours,shuffled_context \
    --taus 0.2,0.4,0.6,0.8,0.9

[run_fig2] OK
  out: /work/paper/source_data/PANCAN_TP53_v1/out


In [42]:
!python paper/scripts/collect_fig2_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70


In [45]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/out/Fig2.risk_coverage.panel.pdf \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
